In [2]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip uninstall h5py
!pip install h5py==2.10.0

     |████████████████████████████████| 412.3 MB 11 kB/s 
     |████████████████████████████████| 3.8 MB 60.6 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 503 kB 67.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e35e0d49faa3b306c69b30004839a50a78bf085f8b543977b55eb935c5f32b3a
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

     |████████████████████████████████| 9.1 MB 5.0 MB/s 
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-3.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 2.9 MB 5.2 MB/s 


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import numpy as np
import pandas as pd
import nltk, re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow_text
from sklearn import preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
ds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Sdataset.csv')

In [21]:
ds.dropna(subset=['Type'], inplace=True)

In [22]:
def deEmojify(text):
    "function to remove emojis from text"
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regrex_pattern.sub(r'',text)
    
for post in ds['Post']:
  deEmojify(post)

In [23]:
nltk.download('stopwords')
stop = stopwords.words('english')
ds['Post'] = ds['Post'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
freq = pd.Series(' '.join(ds['Post']).split()).value_counts()[-10:]
ds['Post'] = ds['Post'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [25]:
ds["Post"]

0      Introducing #Insecure #Comparison #javascript ...
1      I want thread highly recomeneded, Free, browse...
2      The GOSST (Google Open Source Security Team) h...
3      Want learn new @RedHat OpenShift Streams Apach...
4      Thrilled share article "Saga Orchestration #Mi...
                             ...                        
315    The Netherlands long world's tallest nation — ...
316    We know @Schwarzenegger became action star, be...
317    39,000 Americans die gun violence every year. ...
318    While Americans likely eligible Covid-19 vacci...
319    Australia's nuclear-powered submarine deal fue...
Name: Post, Length: 319, dtype: object

In [26]:
embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [27]:
y = list(ds['Type'])
x = list(ds['Post'])
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x), np.asarray(y), test_size=0.2, random_state=42)

In [29]:
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(3, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=16)
    model.save_weights('./elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=16)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Train on 255 samples
255/255 [==============================] - 60s 237ms/sample - loss: 0.7408 - acc: 0.6706


In [30]:
model.get_config()

{'input_layers': [['input_4', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'string',
    'name': 'input_4',
    'ragged': False,
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input_4'},
  {'class_name': 'Lambda',
   'config': {'arguments': DictWrapper({}),
    'dtype': 'string',
    'function': ('4wEAAAAAAAAAAQAAAAcAAABDAAAAcyIAAAB0AHQBoAJ0AaADfAB0AWoEoQKhAWQBZAJkA40DZAEZ\nAFMAKQRO2gdkZWZhdWx0VCkC2glzaWduYXR1cmXaB2FzX2RpY3QpBdoFZW1iZWTaAnRm2gdzcXVl\nZXpl2gRjYXN02gZzdHJpbmcpAdoBeKkAcgoAAAD6HzxpcHl0aG9uLWlucHV0LTI5LTdiMzRhNjVl\nOTQ1ND7aDUVMTW9FbWJlZGRpbmcFAAAAcwIAAAAAAQ==\n',
     None,
     None),
    'function_type': 'lambda',
    'module': '__main__',
    'name': 'lambda_3',
    'output_shape': (1024,),
    'output_shape_module': None,
    'output_shape_type': 'raw',
    'trainable': True},
   'inbound_nodes': [[['input_4', 0, 0, {}]]],
   'name': 'lambda_3'},
  {'class_name': 'Dense',
   'config': {'activation

In [31]:
le = preprocessing.LabelEncoder()
le.fit(y)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_pred= decode(le, predicts)

In [32]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))
from sklearn.metrics import accuracy_score
print("Accuracy of ELMO is:", accuracy_score(y_test,y_pred))

[[ 5 10  0]
 [ 0 29  0]
 [ 0  4 16]]
              precision    recall  f1-score   support

         0.0       1.00      0.33      0.50        15
         1.0       0.67      1.00      0.81        29
         2.0       1.00      0.80      0.89        20

    accuracy                           0.78        64
   macro avg       0.89      0.71      0.73        64
weighted avg       0.85      0.78      0.76        64

Accuracy of ELMO is: 0.78125


In [33]:
from tensorflow.keras.models import save_model
with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)
  model.save('/content/gdrive/MyDrive/Colab Notebooks/my_model.h5')

In [34]:
model_json = model.to_json()
with open("/content/gdrive/MyDrive/Colab Notebooks/model_arch.json", "w") as json_file:
    json_file.write(model_json)

In [35]:
with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)
  model.save_weights("/content/gdrive/MyDrive/Colab Notebooks/my_model_weights.h5")

In [36]:
y_test

array([2., 2., 1., 1., 1., 1., 2., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
       2., 2., 2., 2., 2., 0., 2., 2., 1., 1., 1., 1., 1., 1., 2., 0., 1.,
       2., 0., 2., 1., 0., 0., 2., 0., 2., 1., 0., 1., 0., 2., 1., 1., 1.,
       1., 0., 0., 0., 1., 2., 2., 1., 2., 1., 1., 1., 2.])

In [37]:
y_pred

array([1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 1., 2., 2., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       2., 0., 2., 1., 1., 1., 2., 0., 2., 1., 1., 1., 1., 2., 1., 1., 1.,
       1., 0., 1., 1., 1., 2., 1., 1., 2., 1., 1., 1., 2.])

In [38]:
x_test

array(['Was happy speak friend @ScottMorrisonMP . We reviewed rapid progress India-Australia Comprehensive Strategic Partnership, including recent 2+2 Dialogue. We also exchanged perspectives regional developments forthcoming Quad meeting.',
       'The game simple: win, money yours. If lose, Will play? Squid Game like Battle Royale meets Black Mirror — Netflix',
       'Hello Warriors, The time come Capture The Flag Tamil CTF 2021 happening 10 Days Exciting Prizes HackTheBox, Rootme, Burp Bounty Infosec House. Register now: http://ctf.tamilctf.com',
       'Programming Basics Learn basic computer programming skills master art writing C/C++ programs solve real world problems. https://www.edx.org/course/programming-basics',
       '7 leadership #tech conferences consider fall https://buff.ly/2VRTlz2',
       'Linux Strength Training - I completed room! Check out: https://tryhackme.com/room/linuxstrengthtraining #tryhackme #linux basics #command line #analysis #find #information gatherin

In [40]:
print("Printing all the Job Opportunities")
print(end="\n")
for i in range(len(y_test)):
  if y_pred[i] == 0:
    print(x_test[i])

print(end="\n")
print(end='\n')
print("Printing all the Learning Resources")
print(end='\n')
for i in range(len(y_test)):
  if y_pred[i] == 1:
    print(x_test[i])

Printing all the Job Opportunities

Grafana Labs helps understand metrics dashboards easy create share. Now hiring remote! ⚡️ Frontend Engineer ⚡️ Python Backend Engineer - Cloud ⚡️ Sr Software Engineer Apply 👇nodesk.co/remote-compani...#remotework #remotejobs
New #remote #job Paralegal (Product Privacy) @gitlab https://wantremote.com/job/15219#1631884503
We're hiring (remote onsite) Program Manager Salesforce Research & Insights Operations team! Check job description apply today! salesforce.smh.re/xTA
HackerRank hiring remote Content Marketing Intern https://himalayas.app/companies/hackerrank/jobs/content-marketing-intern
New #remote #job IT Helpdesk Analyst @gitlab https://wantremote.com/job/15218#1631882702


Printing all the Learning Resources

Was happy speak friend @ScottMorrisonMP . We reviewed rapid progress India-Australia Comprehensive Strategic Partnership, including recent 2+2 Dialogue. We also exchanged perspectives regional developments forthcoming Quad meeting.
The game 